In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [2]:
sc = SparkContext(appName="YourTest", master="local[2]")

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [3]:
import random
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.14193344


---

Briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

Suppose the first quadrant is the square with 0 <= x <= 1 and 0 <= y <= 1.

The spark job takes 100000000 random samples of pairs of (x,y) data in the first quadrant using random.random(), where random.random() generates a random float uniformly in the semi-open range [0.0, 1.0)

Out of the 100000000 random samples, count those pairs that satisfy x^2 + y^2 < 1, which are points in the first quadrant that are in the quarter of a circle with radius 1

Now the ratio of all the points that satisfy x^2 + y^2 < 1 over the all the points in the first quadrant with side length = 1, will be roughly the same as the ratio of an area of a circle with radius 1 over a square with sidelength = 2. 

In equation form, this will be

$\frac{\pi r^2}{2^2} \approx \frac{\text{count of all points that satisfy x^2 + y^2 < 1 in first quadrant}}{ \text{all the points in the first quadrant}}$

where r = 1

hence

$\pi \approx 4*\frac{\text{count of all points that satisfy x^2 + y^2 < 1 in first quadrant}}{\text{all the points in the first quadrant}}$

which is what the spark job is doing to estimate  $\pi$

In Steps:

1. sc.parallelize turns range(0,num_samples) into an RDD
2. on top of the RDD, filtered using the function *inside* and only those points that satisfy x^2 + y^2 < 1 were counted.
3. $\pi$ was estimated using the equation $\pi \approx 4*\frac{\text{count of all points that satisfy x^2 + y^2 < 1 in first quadrant}}{\text{all the points in the first quadrant}}$

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [12]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    # read lines
    lines = sc.textFile("Shakespeare.txt")
    # tokenize, map, reduce and count
    count = lines.flatMap(lambda line: simple_tokenize(line)) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x,y: x+y) \
    .count()
    return count

In [6]:
# Test
count_distinct_tokens() 

25975

---

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [13]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    # a pairs helper function
    def pairs(line):
        result = []
        for i in line:
            for j in line:
                if i != j:
                    result.append((i,j))
        return result
    # read file
    lines = sc.textFile("Shakespeare.txt")
    # tokenize, use pairs as keys, map, reduce and count
    count = lines.map(lambda line: simple_tokenize(line))\
                .flatMap(lambda x: pairs(x))\
                .map(lambda word: (word, 1)) \
                .reduceByKey(lambda x,y: x+y) \
                .count()
    return count

In [8]:
# Test
count_distinct_pairs()

1969760

---

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [14]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    # read lines
    lines = sc.textFile("Shakespeare.txt")
    # count total lines
    totalLines = lines.count()
    # tokenize each line, find unique tokens, map, reduce, sort
    count = lines.map(lambda line: simple_tokenize(line))\
                .flatMap(lambda x: set(x))\
                .map(lambda word: (word, (1, 1/totalLines))) \
                .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                .map(lambda x:(x[1][1], x[1][0], x[0]))\
                .sortBy(lambda x: x[0],False)            
    return count.collect()[:50]   

In [10]:
# Test
top_50_tokens_probabilities()

[(0.20091786571727055, 24604, 'and'),
 (0.1984353819269089, 24300, 'the'),
 (0.15235427656832087, 18657, 'i'),
 (0.14892452922637386, 18237, 'to'),
 (0.13575266622027737, 16624, 'of'),
 (0.10844534452629928, 13280, 'a'),
 (0.09959332995803605, 12196, 'you'),
 (0.09430988583841768, 11549, 'my'),
 (0.08667461497003565, 10614, 'in'),
 (0.08630714204054132, 10569, 'that'),
 (0.07150206601447007, 8756, 'is'),
 (0.06720671577193643, 8230, 'not'),
 (0.06167012363422232, 7552, 'with'),
 (0.06039621747864281, 7396, 'me'),
 (0.05982459292165162, 7326, 'for'),
 (0.05836286726877482, 7147, 'it'),
 (0.0544023256953377, 6662, 'be'),
 (0.052466968266668426, 6425, 'this'),
 (0.05228731483447127, 6403, 'his'),
 (0.05089908376749332, 6233, 'your'),
 (0.050670433944696947, 6205, 'but'),
 (0.04749383462084737, 5816, 'he'),
 (0.046889545803456965, 5742, 'have'),
 (0.04132028940534538, 5060, 'thou'),
 (0.04015254209606392, 4917, 'as'),
 (0.0395237550833739, 4840, 'him'),
 (0.03949109082297442, 4836, 'so'),


---

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [3]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    # read file
    lines = sc.textFile("Shakespeare.txt")
    # count total lines
    totalLines = lines.count()
    # pmi helper function
    def pmi(row):
        return (row[0],log((row[1][1])/(row[1][3]*row[1][5]),10),row[1][0], row[1][2], row[1][4])
    # pairs helper function
    def pairs(line):
        result = []
        for i in line:
            for j in line:
                if i != j:
                    result.append((i,j))
        return result
    # single token probabilities
    probabilities_one = lines.map(lambda line: simple_tokenize(line))\
                .flatMap(lambda x: set(x))\
                .map(lambda word: (word, (1, 1/totalLines))) \
                .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
    # pair tokens probabilities
    probabilities_two = lines.map(lambda line: simple_tokenize(line))\
                .map(lambda x: pairs(x)).flatMap(lambda x: set(x)).map(lambda t: (t,1))\
                .reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], x[1], x[1]/totalLines))\
                .filter(lambda x: x[1]>=threshold)
    # combination probabilities
    result = probabilities_two.map(lambda x: (x[0][0], (x[0][1], x[1], x[2])))\
                               .join(probabilities_one)\
                               .map(lambda x: (x[1][0][0], (x[0], x[1][0][1],x[1][0][2], x[1][1])))\
                               .join(probabilities_one)\
                               .map(lambda x: ((x[1][0][0],x[0]), (x[1][0][1],x[1][0][2], x[1][0][3][0], x[1][0][3][1],x[1][1][0],x[1][1][1])))\
                               .map(lambda line: pmi(line))
    return result.collect()

In [4]:
# TEST
PMI(3000)

[(('the', 'of'), 0.34294075191872514, 7266, 24300, 16624),
 (('and', 'of'), 0.028305447826515007, 3565, 24604, 16624),
 (('of', 'and'), 0.028305447826515007, 3565, 16624, 24604),
 (('i', 'and'), -0.05037403533823378, 3338, 18657, 24604),
 (('the', 'and'), 0.045934991832871336, 5427, 24300, 24604),
 (('to', 'and'), 0.01752258803852136, 3815, 18237, 24604),
 (('i', 'to'), 0.04685260592230071, 3095, 18657, 18237),
 (('the', 'to'), 0.051235259829723094, 4072, 24300, 18237),
 (('and', 'to'), 0.01752258803852136, 3815, 24604, 18237),
 (('of', 'the'), 0.34294075191872514, 7266, 16624, 24300),
 (('i', 'the'), -0.08302344185176173, 3058, 18657, 24300),
 (('to', 'the'), 0.051235259829723094, 4072, 18237, 24300),
 (('and', 'the'), 0.045934991832871336, 5427, 24604, 24300),
 (('the', 'i'), -0.08302344185176173, 3058, 24300, 18657),
 (('to', 'i'), 0.04685260592230071, 3095, 18237, 18657),
 (('and', 'i'), -0.05037403533823378, 3338, 24604, 18657)]

In [5]:
# TEST
PMI(6000)

[(('the', 'of'), 0.34294075191872514, 7266, 24300, 16624),
 (('of', 'the'), 0.34294075191872514, 7266, 16624, 24300)]

---

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```


In [6]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    lines = sc.textFile("Shakespeare.txt")
    totalLines = lines.count()
    # pmi helper function
    def pmi(row):
        return (row[0],log((row[1][1])/(row[1][3]*row[1][5]),10),row[1][0], row[1][2], row[1][4])
    # pairs helper function
    def pairs(line):
        result = []
        for i in line:
            for j in line:
                if i != j:
                    result.append((i,j))
        return result
    # helper for return result
    def return_list(x):
        if x[1]:
            return x[1]
        else:
            return x[0]    
    # single token probabilities
    probabilities_one = lines.map(lambda line: simple_tokenize(line))\
            .flatMap(lambda x: set(x))\
            .map(lambda word: (word, (1, 1/totalLines))) \
            .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
    # pair token probabilities
    probabilities_two = lines.map(lambda line: simple_tokenize(line))\
                .map(lambda x: pairs(x)).flatMap(lambda x: set(x)).map(lambda t: (t,1))\
                .reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], x[1], x[1]/totalLines))\
                .filter(lambda x: x[1]>=threshold)
    # combination probabilities
    result = probabilities_two.map(lambda x: (x[0][0], (x[0][1], x[1], x[2])))\
                               .join(probabilities_one)\
                               .map(lambda x: (x[1][0][0], (x[0], x[1][0][1],x[1][0][2], x[1][1])))\
                               .join(probabilities_one)\
                               .map(lambda x: (x[1][0][0], ((x[1][0][0],x[0]), (x[1][0][1],x[1][0][2], x[1][0][3][0], x[1][0][3][1],x[1][1][0],x[1][1][1]))))\
                               .map(lambda line: (line[0], pmi(line[1])))\
                               .groupByKey().mapValues(list)
    # final result
    return result.takeSample(False, samp_size)

    

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.

In [6]:
# TEST
PMI_one_token(2,2)

[('wake',
  [(('wake', 'we'), 0.3542550516083907, 3, 52, 3125),
   (('wake', 'let'), 0.5327153222152776, 3, 52, 2072),
   (('wake', 'when'), 0.76238940934359, 5, 52, 2035),
   (('wake', 'dost'), 1.0487317183971239, 2, 52, 421),
   (('wake', 'king'), 0.4003987533834042, 3, 52, 2810),
   (('wake', 'must'), 0.5086609584483545, 2, 52, 1460),
   (('wake', 'needs'), 1.5056964794846175, 2, 52, 147),
   (('wake', 'pisanio'), 1.6601765895276206, 2, 52, 103),
   (('wake', 'that'), -0.17492882461245507, 3, 52, 10569),
   (('wake', 'to'), 0.1110304204646305, 10, 52, 18237),
   (('wake', 'be'), -0.15059081412234246, 2, 52, 6662),
   (('wake', 'and'), 0.23625060532031097, 18, 52, 24604),
   (('wake', 'you'), -0.11217360579661026, 4, 52, 12196),
   (('wake', 'or'), 0.31393758817353196, 2, 52, 2286),
   (('wake', 'if'), 0.43635061552939597, 4, 52, 3449),
   (('wake', 'some'), 0.568185410579138, 2, 52, 1273),
   (('wake', 'so'), -0.011472477955922775, 2, 52, 4836),
   (('wake', 'softly'), 2.33059113341

In [7]:
# TEST
PMI_one_token(1000,3)

[('of',
  [(('of', 'to'), -0.09813466615126466, 1975, 16624, 18237),
   (('of', 'that'), 0.03856379243792773, 1568, 16624, 10569),
   (('of', 'and'), 0.028305447826515007, 3565, 16624, 24604),
   (('of', 'you'), -0.060904989766341754, 1439, 16624, 12196),
   (('of', 'this'), 0.1517481996162481, 1237, 16624, 6425),
   (('of', 'in'), 0.052760101633904634, 1627, 16624, 10614),
   (('of', 'the'), 0.34294075191872514, 7266, 16624, 24300),
   (('of', 'his'), 0.0799844206133649, 1045, 16624, 6403),
   (('of', 'i'), -0.08531809933120071, 2081, 16624, 18657),
   (('of', 'is'), 0.020115230526300435, 1245, 16624, 8756),
   (('of', 'a'), 0.1355179687974917, 2463, 16624, 13280),
   (('of', 'my'), 0.01180279106315552, 1611, 16624, 11549)]),
 ('but',
  [(('but', 'the'), -0.02180400215716167, 1171, 6205, 24300),
   (('but', 'i'), 0.16991073624528089, 1398, 6205, 18657)]),
 ('and',
  [(('and', 'of'), 0.028305447826515007, 3565, 24604, 16624),
   (('and', 'have'), -0.05220611059650287, 1023, 24604, 5742

In [46]:
# TEST
PMI_one_token(1,1)

[('stinkingly',
  [(('stinkingly', 'so'), 1.403500870014895, 1, 1, 4836),
   (('stinkingly', 'go'), 1.8647508891006104, 1, 1, 1672),
   (('stinkingly', 'mend'), 3.3245591686406737, 1, 1, 58),
   (('stinkingly', 'depending'), 4.184897175211667, 1, 1, 8)])]

In [47]:
# TEST
PMI_one_token(2,4)

[('fighteth',
  [(('fighteth', 'as'), 1.396286953913468, 2, 2, 4917),
   (('fighteth', 'he'), 1.3233627643526504, 2, 2, 5816)]),
 ('porn',
  [(('porn', 'you'), 0.8256784874545453, 2, 3, 12196),
   (('porn', 'he'), 1.1472715052969693, 2, 3, 5816),
   (('porn', 'at'), 1.520784289445131, 2, 3, 2461),
   (('porn', 'tell'), 1.888231984950139, 2, 3, 1056),
   (('porn', 'fluellen'), 3.138597155558698, 3, 3, 89),
   (('porn', 'is'), 0.9695901502520373, 2, 3, 8756)]),
 ('valor',
  [(('valor', 'to'), 0.28296571974915397, 2, 7, 18237),
   (('valor', 'and'), 0.4539433951655096, 4, 7, 24604),
   (('valor', 'the'), 0.15831284425492706, 2, 7, 24300),
   (('valor', 'with'), 0.8419483956190277, 3, 7, 7552),
   (('valor', 'his'), 0.7375356160291898, 2, 7, 6403)]),
 ('pinch',
  [(('pinch', 'like'), 0.8008351210781057, 3, 33, 1761),
   (('pinch', 'let'), 0.5541134669165081, 2, 33, 2072),
   (('pinch', 'no'), 0.320397218642125, 2, 33, 3549),
   (('pinch', 'as'), 0.5767430183715995, 5, 33, 4917),
   (('pinc